In [16]:
from qtpg.team import Team
from qtpg.agent import Agent
from qtpg.trainer import Trainer
from qtpg.figure13 import Figure13

In [17]:
# gp params
numLearners = 4
numAgents = 2
numRuns = 1
numGens = 50
numEpisodes = 20
gap = 0.5
# rl params
alpha, discount, epsilon = 0.1, 0.9, 0.1
# env params
memorySize = 20
legalMove = 0.1
illegalMove = -0.01
outOfBounds = -0.01
memoryRepeat = -0.01
goalReached = 100

In [18]:
for run in range(numRuns):
    # init agent population
    trainer = Trainer(gap, numLearners, numAgents, alpha, discount, epsilon)
    trainer.createInitAgents()
    # init env
    env = Figure13(5, 5, (2, 4), (4, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
    champs = []
    for gen in range(numGens):
        for agent in trainer.agents:
            episode_fitness = [] # new
            for ep in range(numEpisodes):
                if agent.win is not True:
                    env.reset()
                    fitness, win = agent.evaluate_fitness(env)
                    episode_fitness.append(fitness)
                    if win == True:
                        agent.win = True
                        champs.append(agent)
                        print('win!')
            # average fitness over episodes...
            fitness = 0
            for i in range(len(episode_fitness)):
                fitness += episode_fitness[i]
            fitness /= len(episode_fitness)+1
            print(fitness)
            agent.fitness = fitness
        trainer.evolve()

winner!! start
c1561359-5a6d-4fcb-8390-7c55a01f85c5 0 0.01852402698945738
c1561359-5a6d-4fcb-8390-7c55a01f85c5 1 10.037101592691368
f2388bd9-7032-4c01-97b9-96b2ca1a285f 3 0.0012628278268485079
f2388bd9-7032-4c01-97b9-96b2ca1a285f 2 0.022320896850021768
9fdc2003-e2ce-488f-852b-c6cf13e2fc93 3 0.012946301563207991
9fdc2003-e2ce-488f-852b-c6cf13e2fc93 2 0.026022756403284827
f90fd327-bc82-487d-b84f-6b79923f5373 3 0.024366331522130122
f90fd327-bc82-487d-b84f-6b79923f5373 2 0.05463515354365283
winner!! end
win!
25.464999999999996
winner!! start
2bd33123-c052-4540-bc9a-9ac206da9287 2 0.02564591289020549
2bd33123-c052-4540-bc9a-9ac206da9287 3 0.01930444880402165
031c57b4-a30b-4637-80bf-46c71b80ef3f 1 10.037239686101158
031c57b4-a30b-4637-80bf-46c71b80ef3f 0 0.019034555333874277
0966151c-ead6-4bce-869d-ad3a8ccf0240 1 -0.0028136719094211957
0966151c-ead6-4bce-869d-ad3a8ccf0240 0 0.024083848070587208
7cc50873-67d2-4bc8-878f-6d06744ed765 0 0
7cc50873-67d2-4bc8-878f-6d06744ed765 1 -0.000930480600340

winner!! start
2bd33123-c052-4540-bc9a-9ac206da9287 2 2.2519800593280244
2bd33123-c052-4540-bc9a-9ac206da9287 3 1.9892249402560909
031c57b4-a30b-4637-80bf-46c71b80ef3f 1 12.118198263465832
031c57b4-a30b-4637-80bf-46c71b80ef3f 0 2.329299483116656
0966151c-ead6-4bce-869d-ad3a8ccf0240 1 2.4946657659072096
0966151c-ead6-4bce-869d-ad3a8ccf0240 0 1.9521244697946767
7cc50873-67d2-4bc8-878f-6d06744ed765 0 2.2789310714236937
7cc50873-67d2-4bc8-878f-6d06744ed765 1 2.4257657286541643
winner!! end
win!
20.431999999999995
0.0
winner!! start
2bd33123-c052-4540-bc9a-9ac206da9287 2 1.5712336542446455
2bd33123-c052-4540-bc9a-9ac206da9287 3 1.5706473012805193
031c57b4-a30b-4637-80bf-46c71b80ef3f 1 11.455023186564759
031c57b4-a30b-4637-80bf-46c71b80ef3f 0 1.587699151368292
0966151c-ead6-4bce-869d-ad3a8ccf0240 1 1.5996290468227845
0966151c-ead6-4bce-869d-ad3a8ccf0240 0 1.5987829797269637
7cc50873-67d2-4bc8-878f-6d06744ed765 0 1.7246854032040628
7cc50873-67d2-4bc8-878f-6d06744ed765 1 1.7260272419195763
win

winner!! start
2bd33123-c052-4540-bc9a-9ac206da9287 2 2.97868873541245
2bd33123-c052-4540-bc9a-9ac206da9287 3 2.98478230015204
031c57b4-a30b-4637-80bf-46c71b80ef3f 1 2.973859435808808
031c57b4-a30b-4637-80bf-46c71b80ef3f 0 2.990161602497674
0966151c-ead6-4bce-869d-ad3a8ccf0240 1 3.0788094531182915
0966151c-ead6-4bce-869d-ad3a8ccf0240 0 3.048975930124818
7cc50873-67d2-4bc8-878f-6d06744ed765 0 3.0780162685435952
7cc50873-67d2-4bc8-878f-6d06744ed765 1 12.799967105355414
winner!! end
win!
18.216666666666665
0.0
winner!! start
2bd33123-c052-4540-bc9a-9ac206da9287 2 2.275803599104416
2bd33123-c052-4540-bc9a-9ac206da9287 3 2.2868404748529203
031c57b4-a30b-4637-80bf-46c71b80ef3f 1 12.0903391631553
031c57b4-a30b-4637-80bf-46c71b80ef3f 0 2.298076821189999
0966151c-ead6-4bce-869d-ad3a8ccf0240 1 2.369047803413911
0966151c-ead6-4bce-869d-ad3a8ccf0240 0 2.3735994051535765
7cc50873-67d2-4bc8-878f-6d06744ed765 0 2.506375956820497
7cc50873-67d2-4bc8-878f-6d06744ed765 1 2.4817954993735487
winner!! end
w

In [19]:
print(len(champs))
env.reset()
champ_index = 0
champs[champ_index].team.epsilon = 0.0

for q_value in champs[champ_index].team.q_table:
    print(str(q_value['learner']) + ' ' + str(q_value['action']) + ' ' + str(q_value['q']))

for i in range(100):
    score, win, states = champs[champ_index].replay(env)
    print(win)
    print(score)

51
c1561359-5a6d-4fcb-8390-7c55a01f85c5 0 0.01852402698945738
c1561359-5a6d-4fcb-8390-7c55a01f85c5 1 10.037101592691368
f2388bd9-7032-4c01-97b9-96b2ca1a285f 3 0.0012628278268485079
f2388bd9-7032-4c01-97b9-96b2ca1a285f 2 0.022320896850021768
9fdc2003-e2ce-488f-852b-c6cf13e2fc93 3 0.012946301563207991
9fdc2003-e2ce-488f-852b-c6cf13e2fc93 2 0.026022756403284827
f90fd327-bc82-487d-b84f-6b79923f5373 3 0.024366331522130122
f90fd327-bc82-487d-b84f-6b79923f5373 2 0.05463515354365283
False
-1.520000000000001
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
False
-2.0000000000000013
F